## Loading Dataset

In [40]:
import pickle
import pandas as pd
import numpy as np

from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model, metrics, svm, neighbors, gaussian_process, ensemble, neural_network
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [55]:
df = pickle.load(open("datafiles/after-EDA.ft", "rb"))
df.head()

,totPurchaseAmt,latitude,longitude,bathrooms,bedrooms,livingArea,yearBuilt,averageSchoolRating,zip,zestimate,sentiment
1,198000.0,45.237189,-93.409535,2.0,4.0,1716.0,1996,5.333333,55303,285985.0,0.4939
3,415000.0,45.278217,-93.407533,2.0,3.0,3108.0,1985,6.333333,55303,431995.0,0.2617
4,329900.0,45.143781,-93.021604,3.0,4.0,1814.0,2001,6.500000,55038,318162.0,0.2732
5,262000.0,45.164166,-93.297836,2.0,3.0,2158.0,1985,4.666667,55043,272169.0,0.9136
6,280000.0,45.283700,-93.332023,2.0,3.0,1993.0,1976,7.333333,55304,290365.0,0.9428


In [58]:
len(df)

26789

In [57]:
grouped = df.groupby('zip')
df = grouped.filter(lambda x: x['zip'].count() > 70)

## Converting Categorical Variables 

In [59]:
df = pd.get_dummies(data=df, columns=["bathrooms", "bedrooms", "zip"])
df.head()

,totPurchaseAmt,latitude,longitude,livingArea,yearBuilt,averageSchoolRating,zestimate,sentiment,bathrooms_1.0,bathrooms_1.2,...,zip_55437,zip_55441,zip_55442,zip_55443,zip_55444,zip_55445,zip_55446,zip_55447,zip_55448,zip_55449
1,198000.0,45.237189,-93.409535,1716.0,1996,5.333333,285985.0,0.4939,0,0,...,0,0,0,0,0,0,0,0,0,0
3,415000.0,45.278217,-93.407533,3108.0,1985,6.333333,431995.0,0.2617,0,0,...,0,0,0,0,0,0,0,0,0,0
4,329900.0,45.143781,-93.021604,1814.0,2001,6.500000,318162.0,0.2732,0,0,...,0,0,0,0,0,0,0,0,0,0
6,280000.0,45.283700,-93.332023,1993.0,1976,7.333333,290365.0,0.9428,0,0,...,0,0,0,0,0,0,0,0,0,0
7,237000.0,45.105931,-93.261656,2058.0,1959,3.666667,246844.0,0.9869,0,0,...,0,0,0,0,0,0,0,0,0,0


## Train Test Split 

In [60]:
X = df.drop(columns=["totPurchaseAmt", "latitude", "longitude"])
y = df["totPurchaseAmt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train_zestimate = X_train["zestimate"]
X_train = X_train.drop(columns=["zestimate"])

X_test_zestimate = X_test["zestimate"]
X_test = X_test.drop(columns=["zestimate"])

In [61]:
scaler = StandardScaler()
scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  # apply same transformation to test data

## Models

In [62]:
cnn_model = Sequential()

n_cols = X_train.shape[1]
cnn_model.add(Dense(16, activation='relu', input_shape=(n_cols,)))
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dense(16, activation='relu'))
cnn_model.add(Dense(1))

cnn_model.compile(optimizer='adam', loss='mean_absolute_error', metrics=["mae"])

In [64]:
cnn_model.fit(X_train, y_train, validation_split=0.1, epochs=100, verbose=1)

Epoch 1/100
528/528 [==============================] - 3s 6ms/step - loss: 101613.4531 - mae: 101613.4531 - val_loss: 67089.0469 - val_mae: 67089.0469
Epoch 2/100
528/528 [==============================] - 3s 6ms/step - loss: 61632.3242 - mae: 61632.3242 - val_loss: 63095.7227 - val_mae: 63095.7227
Epoch 3/100
528/528 [==============================] - 3s 5ms/step - loss: 58820.8867 - mae: 58820.8867 - val_loss: 61500.8555 - val_mae: 61500.8555
Epoch 4/100
528/528 [==============================] - 3s 6ms/step - loss: 58246.1797 - mae: 58246.1797 - val_loss: 63786.1289 - val_mae: 63786.1289
Epoch 5/100
528/528 [==============================] - 3s 6ms/step - loss: 57470.8594 - mae: 57470.8594 - val_loss: 62653.6914 - val_mae: 62653.6914
Epoch 6/100
528/528 [==============================] - 3s 6ms/step - loss: 56896.6797 - mae: 56896.6797 - val_loss: 61491.9453 - val_mae: 61491.9453
Epoch 7/100
528/528 [==============================] - 3s 6ms/step - loss: 56599.4922 - mae: 56599.4922 

528/528 [==============================] - 3s 6ms/step - loss: 52144.0820 - mae: 52144.0820 - val_loss: 61351.8867 - val_mae: 61351.8867
Epoch 56/100
528/528 [==============================] - 3s 6ms/step - loss: 52231.5664 - mae: 52231.5664 - val_loss: 62280.5625 - val_mae: 62280.5625
Epoch 57/100
528/528 [==============================] - 3s 6ms/step - loss: 52241.4805 - mae: 52241.4805 - val_loss: 62753.4258 - val_mae: 62753.4258
Epoch 58/100
528/528 [==============================] - 3s 5ms/step - loss: 51866.5742 - mae: 51866.5742 - val_loss: 60533.4688 - val_mae: 60533.4688
Epoch 59/100
528/528 [==============================] - 3s 5ms/step - loss: 51845.4727 - mae: 51845.4727 - val_loss: 63091.2227 - val_mae: 63091.2227
Epoch 60/100
528/528 [==============================] - 3s 6ms/step - loss: 51855.9141 - mae: 51855.9141 - val_loss: 60880.5156 - val_mae: 60880.5156
Epoch 61/100
528/528 [==============================] - 3s 6ms/step - loss: 51815.2461 - mae: 51815.2461 - val_lo

In [65]:
models = {
    #"Neural Network" : cnn_model,
    "Linear Regression" : linear_model.LinearRegression(), 
    #"Support Vector Machine" : svm.SVR(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Ridge Regression" : linear_model.Ridge(alpha=.5),
    "Least Angle Regression" : linear_model.LassoLars(alpha=.1),
    "Bayesian Ridge Regression" : linear_model.BayesianRidge(),
    "SGD Regressor" : linear_model.SGDRegressor(),
    "Nearest Neighbors Regression" : neighbors.KNeighborsRegressor(),
    #"Gaussian Process" : gaussian_process.GaussianProcessRegressor(),
    "Random Forest" : ensemble.RandomForestRegressor(),
    'Gradient Boosting Regressor' : ensemble.GradientBoostingRegressor()
}

for model_name in models:
    model = models[model_name]

    if model_name == "Neural Network":
        model.fit(X_train, y_train, validation_split=0.1, epochs=100, verbose=0)
    else:
        model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    print ("=== " + model_name + " ===")
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('\n')

=== Linear Regression ===
Mean Absolute Error: 309403437005579.5
Mean Squared Error: 3.846929611762503e+32
Root Mean Squared Error: 1.961359123608551e+16


=== Decision Tree ===
Mean Absolute Error: 78685.99993239601
Mean Squared Error: 22522082270.635963
Root Mean Squared Error: 150073.58951739632


=== Ridge Regression ===
Mean Absolute Error: 61904.44350288182
Mean Squared Error: 13392450661.323738
Root Mean Squared Error: 115725.75625729882


=== Least Angle Regression ===
Mean Absolute Error: 61892.11067172486
Mean Squared Error: 13392633901.665377
Root Mean Squared Error: 115726.54795536492


=== Bayesian Ridge Regression ===
Mean Absolute Error: 61813.5531835889
Mean Squared Error: 13400914624.470406
Root Mean Squared Error: 115762.3195364986


=== SGD Regressor ===
Mean Absolute Error: 3396843622.0418334
Mean Squared Error: 9.427794303313826e+21
Root Mean Squared Error: 97096829522.46086


=== Nearest Neighbors Regression ===
Mean Absolute Error: 67588.08312181162
Mean Squared 

In [66]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 20)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = ensemble.RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 